# Ratio Approach

In [1]:
import os
os.chdir("../../")

import numpy as np
import pandas as pd

from scripts.python.tsa.mtsmodel import *
from scripts.python.tsa.ts_eval import *

In [4]:
for country in ["palau", "samoa", "tonga"]:
    
    
    mod = RatioPipe(country=country, data=None)
    print(f"\n{country.upper()}'s Official and Avaition Data.")
    mod.read_and_merge()
    
    print(f"\n Create Loaded Factor for {country.upper()}:")
    mod.transform()
    
    print(f"\n Regression {country.upper()}:")
    mod.fit()
    pred_df = mod.get_prediction_df()

    mean_pred, naive_pred = \
        mean_method(pred_df["total"]), naive_method(pred_df["total"])
    
    benchmark = pd.DataFrame()
    eval_name = ["mean", "naive", "ratio"]
    for idx, pred in zip(eval_name, [mean_pred, naive_pred, pred_df["pred_mean"]]):
        eval_df = pd.DataFrame(calculate_evaluation(pred_df["total"], pred), index=[idx])
        benchmark = pd.concat([benchmark, eval_df])
    display(benchmark)


PALAU's Official and Avaition Data.


,date,seats_arrivals_total,seats_arrivals_intl,number_of_flights_total,number_of_flights_intl,japan,south_korea,taiwan,china,usa/canada,europe,others,total,stringency_index,covid,palau_flights,palau_hotel,palau_travel
0,2019-01-01,13048,13048,75,75,2055.0,1035.0,1092.0,4059.0,702.0,438.0,371.0,9752.0,0.0,0.0,0.493228,2.559978,0.194222
1,2019-02-01,13281,13281,74,74,2434.0,1090.0,1190.0,2549.0,826.0,483.0,461.0,9033.0,0.0,0.0,0.310671,2.841414,0.282245
2,2019-03-01,12870,12870,73,73,1756.0,808.0,1099.0,3182.0,777.0,368.0,415.0,8405.0,0.0,0.0,0.386389,2.921431,0.286529
3,2019-04-01,10806,10806,64,64,1288.0,1095.0,1393.0,2489.0,607.0,185.0,421.0,7478.0,0.0,0.0,0.293529,2.981907,0.338945
4,2019-05-01,11472,11472,67,67,650.0,655.0,1399.0,2321.0,622.0,119.0,305.0,6071.0,0.0,0.0,0.306351,3.537987,0.168664



 Create Loaded Factor for PALAU:
Abnormal value produced with a value of inf.
Abnormal value produced with a value of 1.7439759036144578.

 Regression PALAU:
                            WLS Regression Results                            
Dep. Variable:                  ratio   R-squared:                       0.553
Model:                            WLS   Adj. R-squared:                  0.466
Method:                 Least Squares   F-statistic:                     6.366
Date:                Wed, 10 May 2023   Prob (F-statistic):           6.66e-05
Time:                        00:27:14   Log-Likelihood:                 13.430
No. Observations:                  44   AIC:                            -10.86
Df Residuals:                      36   BIC:                             3.414
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                             coef    std err       

,date,ratio,total,seats_arrivals_intl,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper,pred_mean
0,2019-01-01,0.747394,9752.0,13048,0.586118,0.082216,0.419377,0.752860,0.152919,1.019317,7647.672011
1,2019-02-01,0.680145,9033.0,13281,0.650264,0.075176,0.497800,0.802728,0.222358,1.078171,8636.159406
2,2019-03-01,0.653069,8405.0,12870,0.653386,0.075026,0.501227,0.805545,0.225588,1.081184,8409.074189
3,2019-04-01,0.692023,7478.0,10806,0.568263,0.076326,0.413466,0.723060,0.139519,0.997006,6140.645641
4,2019-05-01,0.529202,6071.0,11472,0.444172,0.093864,0.253807,0.634536,0.001343,0.887000,5095.536158


,MSE,RMSE,MAE,SMAPE
mean,1.198414e+07,3461.811598,3054.239669,131.748643
naive,1.192288e+06,1091.919475,636.883721,54.461211
ratio,1.381506e+06,1175.374983,636.212758,55.636188


...


TONGA's Official and Avaition Data.


,date,seats_arrivals_total,seats_arrivals_intl,number_of_flights_total,number_of_flights_intl,air,ship,yacht,total,stringency_index,covid,tonga_flights,tonga_hotel,tonga_travel
0,2019-01-01,17698,14706,102,68,2709.0,3448.0,0.0,6157.0,0.0,0.0,1.075798,0.980824,0.203206
1,2019-02-01,11440,9740,80,45,3109.0,5570.0,7.0,8686.0,0.0,0.0,0.940670,0.926124,0.230846
2,2019-03-01,12662,10236,107,55,4183.0,3508.0,13.0,7704.0,0.0,0.0,0.852194,1.218091,0.268547
3,2019-04-01,10919,10615,78,53,5166.0,1506.0,128.0,6800.0,0.0,0.0,0.793732,1.198991,0.132975
4,2019-05-01,12226,11858,87,64,6647.0,1798.0,489.0,8934.0,0.0,0.0,0.976347,1.162232,0.247394



 Create Loaded Factor for TONGA:
Abnormal value produced with a value of 0.0.
Abnormal value produced with a value of 0.0.
Abnormal value produced with a value of 0.0.
Abnormal value produced with a value of 0.0.
Abnormal value produced with a value of 0.0.

 Regression TONGA:
                            WLS Regression Results                            
Dep. Variable:                  ratio   R-squared:                       0.887
Model:                            WLS   Adj. R-squared:                  0.859
Method:                 Least Squares   F-statistic:                     31.50
Date:                Wed, 10 May 2023   Prob (F-statistic):           1.19e-11
Time:                        00:27:15   Log-Likelihood:                 30.796
No. Observations:                  36   AIC:                            -45.59
Df Residuals:                      28   BIC:                            -32.92
Df Model:                           7                                         
Covariance

,date,ratio,total,seats_arrivals_intl,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper,pred_mean
0,2019-01-01,0.418673,6157.0,14706,0.629559,0.053745,0.519467,0.739652,0.366501,0.892618,9258.301869
1,2019-02-01,0.891786,8686.0,9740,0.624753,0.051336,0.519597,0.729909,0.363722,0.885784,6085.091917
2,2019-03-01,0.752638,7704.0,10236,0.618197,0.049301,0.517209,0.719184,0.358817,0.877577,6327.860469
3,2019-04-01,0.640603,6800.0,10615,0.670236,0.060432,0.546448,0.794025,0.401158,0.939315,7114.559876
4,2019-05-01,0.753415,8934.0,11858,0.650339,0.049443,0.549059,0.751618,0.390845,0.909832,7711.714436


,MSE,RMSE,MAE,SMAPE
mean,1.395667e+07,3735.862783,3458.520088,157.537361
naive,1.765922e+06,1328.880074,625.138889,64.875571
ratio,1.690720e+06,1300.277050,687.109494,98.439777
